In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv("/content/train_8.csv")
test = pd.read_csv("/content/test_8.csv")

In [5]:
train.head()

,x1,x2,x3,f
0,0.455058,0.146759,2.426221,D
1,0.403961,1.402071,2.891587,B
2,0.351298,0.095268,2.263430,C
3,0.664298,0.600407,0.670098,A
4,0.463471,1.633033,1.037303,D


In [16]:
X = np.array(train[["x1", "x2", "x3"]])
y = np.array(train["f"])
X_test = np.array(test[["x1", "x2", "x3"]])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [19]:
def label2id(x):
    x[x == 'A'] = 0
    x[x == 'B'] = 1
    x[x == 'C'] = 2
    x[x == 'D'] = 3

In [25]:
def id2label(x):
    x[x == 0] = 'A'
    x[x == 1] = 'B'
    x[x == 2] = 'C'
    x[x == 3] = 'D'

In [20]:
label2id(y_train)
label2id(y_val)

# LogisticRegression

In [21]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train_scaled, y_train.astype(np.float64))
clf.score(X_train_scaled, y_train.astype(np.float64))

0.9428571428571428

In [26]:
answer = clf.predict(X_test_scaled)
ans = pd.DataFrame(answer)
id2label(ans)
ans.to_csv("test.csv", index=False)

# ConvNet

In [28]:
class ConvNet(torch.nn.Module):

    def __init__(self, n_classes=4, dropout_p=0.0):
        '''
        :param int n_classes: Число выходных признаков
        :param bool use_batchnorm: Использовать ли батчнорм между свёрточными слоями
        :param float dropout_p: Вероятность обнуления активации слоем Dropout
        '''
        super().__init__()

        self.n_classes = n_classes

        ### your code here
        self.dropout_p = dropout_p
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(3, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=dropout_p),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=dropout_p),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=dropout_p),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=dropout_p),
            torch.nn.Linear(512, self.n_classes)
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        classifier_layer = self.classifier(x)
        return classifier_layer

In [29]:
dense_network = ConvNet()
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(dense_network.parameters(), lr=2e-2)

In [30]:
device = torch.device('cpu')
dense_network.to(device)

ConvNet(
  (classifier): Sequential(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.0, inplace=False)
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.0, inplace=False)
    (9): Linear(in_features=512, out_features=512, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.0, inplace=False)
    (12): Linear(in_features=512, out_features=4, bias=True)
  )
)

In [31]:
import tqdm.notebook as tqdm
from functools import partial

In [32]:
import tqdm.notebook as tqdm

def training_loop(n_epochs, network, loss_fn, optimizer, ds_train, ds_test, device):
    '''
    :param int n_epochs: Число итераций оптимизации
    :param torch.nn.Module network: Нейронная сеть
    :param Callable loss_fn: Функция потерь
    :param torch.nn.Optimizer optimizer: Оптимизатор
    :param Tuple[torch.Tensor, torch.Tensor] ds_train: Признаки и метки истинного класса обучающей выборки
    :param Tuple[torch.Tensor, torch.Tensor] ds_test: Признаки и метки истинного класса тестовой выборки
    :param torch.Device device: Устройство на котором будут происходить вычисления
    :returns: Списки значений функции потерь и точности на обучающей и тестовой выборках после каждой итерации
    '''

    train_losses, test_losses, train_accuracies, test_accuracies = [], [], [], []
    for epoch in tqdm.tqdm(range(n_epochs), total=n_epochs, leave=True):
        # Переводим сеть в режим обучения
        ### your code here

        network = network.train()
        # Итерация обучения сети
        def closure():
            '''
            Функция-замыкания для подсчёта градиентов функции потерь по обучающей выборке:
                1. Отчистка текущих градиентов
                2. Выполнение прямого прохода по сети в вычисление функции потерь
                3. Вычисление градиентов функции потерь
            :returns: Значение функции потерь
            '''
            optimizer.zero_grad() # зануление градиентов
            network_out = network(ds_train[0].to(device)) #проход
            loss = loss_fn(network_out, ds_train[1].to(device))
            train_losses.append(loss.item())
            train_accuracies.append((torch.argmax(network_out, dim=1) == ds_train[1]).float().mean().item()*100)
            loss.backward()
            return loss

            ### your code here

        # Шаг оптимизации
        ### your code here
        optimizer.step(closure)
        # Переводим сеть в инференс режим
        ### your code here
        network.eval()
        # При тестировании сети нет необходимости считать градиенты, поэтому можно отключить автоматическое дифференцирование
        #   для ускорения операций
        with torch.no_grad():
            # Вычисление качества и функции потерь на обучающей выборке
            ### your code here
            network_out = network(ds_test[0].to(device))
            loss = loss_fn(network_out, ds_test[1].to(device))
            # Вычисление качества и функции потерь на тестовой выборке
            ### your code here
            test_losses.append(loss.item())
            test_accuracies.append((torch.argmax(network_out, dim=1) == ds_test[1].to(device)).float().mean().item()*100)
            if epoch % 1 == 0:
                print(
                    'Loss (Train/Test): {0:.3f}/{1:.3f}. Accuracy, % (Train/Test): {2:.2f}/{3:.2f}'.format(
                        train_losses[-1], test_losses[-1], train_accuracies[-1], test_accuracies[-1]
                    )
                )

    return train_losses, test_losses, train_accuracies, test_accuracies

In [33]:
ds_train=(torch.from_numpy(X_train_scaled.astype(np.float32)), torch.from_numpy(y_train.astype(np.float32)).type(torch.LongTensor))
ds_test=(torch.from_numpy(X_val_scaled.astype(np.float32)), torch.from_numpy(y_val.astype(np.float32)).type(torch.LongTensor))

In [ ]:
train_losses, test_losses, train_accs, test_accs = training_loop(
    n_epochs=800, network=dense_network, loss_fn=loss_fn,
    optimizer=optimizer, ds_train=ds_train, ds_test=ds_test, device=device
)

In [35]:
y_test = dense_network(torch.from_numpy(X_test_scaled.astype(np.float32)))
answer = torch.argmax(y_test, dim=1).numpy()

In [38]:
y_pred = dense_network(torch.from_numpy(X_val_scaled.astype(np.float32)))
y_pred = torch.argmax(y_pred, dim=1).numpy()

In [41]:
(y_pred == y_val).sum() / len(y_pred)

0.9555555555555556

In [36]:
ans = pd.DataFrame(answer)
id2label(ans)
ans.to_csv("testconv.csv", index=False)